# Part 1. ETF 목록 수집하기
https://github.com/JerBouma/FinanceDatabase

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import financedatabase as fd

In [ ]:
etfs = fd.ETFs()

In [ ]:
all_etfs = etfs.search()
print(len(all_etfs))
all_etfs.head()

In [ ]:
# summary가 결측인 데이터를 삭제
all_etfs = all_etfs.dropna()
print(len(all_etfs))
all_etfs.head()

# Part 2. ETF holdings 보유내역 수집

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO

symbol = "QQQ"
url = f"https://www.schwab.wallst.com/schwab/Prospect/research/etfs/schwabETF/index.asp?type=holdings&symbol={symbol}"

response = requests.get(url)
df_list = pd.read_html(StringIO(response.text))

len(df_list)

In [ ]:
df_list[0]

In [ ]:
# LastPrice, Date 추출
last_price = df_list[0]["Last Price"].iloc[0]
date = df_list[0]["Last Price"].iloc[1]

last_price, date

In [ ]:
last_price = float(last_price.replace("$", ""))
date = date.replace("As of close\xa0", "")

last_price, date

In [ ]:
df_list[1]

In [ ]:
# 데이터를 추출하는 함수
def get_schwab_etf_holdings(symbol):
    url = f"https://www.schwab.wallst.com/schwab/Prospect/research/etfs/schwabETF/index.asp?type=holdings&symbol={symbol}"
    response = requests.get(url)
    df_list = pd.read_html(StringIO(response.text))
    df_holdings = df_list[1]
    last_price = df_list[0]["Last Price"].iloc[0]
    date = df_list[0]["Last Price"].iloc[1]
    last_price = float(last_price.replace("$", ""))
    date = date.replace("As of close\xa0", "")
    return df_holdings, last_price, date


# 함수 실행
symbol = "QQQ"
df_holdings, last_price, date = get_schwab_etf_holdings(symbol)
print(last_price, date)
df_holdings.head()

# Part 3. LangChain으로 추천 시스템 만들기

In [ ]:
all_etfs.head(2)

In [ ]:
all_etfs = all_etfs.reset_index()
all_etfs.head()

In [ ]:
from langchain_community.document_loaders import DataFrameLoader

loader = DataFrameLoader(all_etfs, page_content_column="summary")

docs = loader.load()
len(docs)

In [ ]:
docs[0].page_content

In [ ]:
docs[0].metadata

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [ ]:
# 벡터스토어에 저장 - 테스트를 위해서 10개만 저장

from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(docs[:10], OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY))

In [ ]:
# 벡터스토어에 저장된 문서를 검색
from langchain.chains.query_constructor.base import AttributeInfo, StructuredQueryOutputParser, get_query_constructor_prompt
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

# 문서 객체의 메타데이터에 대한 설명
metadata_field_info = [
    AttributeInfo(
        name="symbol",
        description="The stock symbol for the ETF, uniquely identifying the fund on the exchange.",
        type="string",
    ),
    AttributeInfo(
        name="name",
        description="The official name of the ETF, which describes the ETF's investment focus.",
        type="string",
    ),
    AttributeInfo(
        name="currency",
        description="The currency in which the ETF is denominated.",
        type="string",
    ),
    AttributeInfo(
        name="category_group",
        description="The general category or type of equities the ETF invests in, such as 'Equities'.",
        type="string",
    ),
    AttributeInfo(
        name="category",
        description="The specific investment category of the ETF, for example, 'Emerging Markets'.",
        type="string",
    ),
    AttributeInfo(
        name="family",
        description="The fund family or asset management company that manages the ETF, e.g., 'BlackRock Asset Management'.",
        type="string",
    ),
    AttributeInfo(
        name="exchange",
        description="The stock exchange where the ETF is listed, such as 'NMS' (NASDAQ Market System).",
        type="string",
    ),
    AttributeInfo(
        name="market",
        description="The regional market where the ETF is primarily traded, such as 'us_market'.",
        type="string",
    ),
]

# 문서 객체의 내용에 대한 설명
document_content_description = "Brief summary of a financial ETF, including its investment focus and strategy."

# LLM 정의
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, api_key=OPENAI_API_KEY)


# 검색 쿼리 생성기
prompt = get_query_constructor_prompt(
    document_content_description,
    metadata_field_info,
)

output_parser = StructuredQueryOutputParser.from_components()
query_constructor = prompt | llm | output_parser

query_constructor.invoke({"query": "Find a fund that invests in equities of clean energy production companies."})

In [ ]:
from langchain.retrievers.self_query.chroma import ChromaTranslator

# SelfQueryRetriever 생성
retriever = SelfQueryRetriever(
    query_constructor=query_constructor,
    vectorstore=vectorstore, 
    structured_query_translator=ChromaTranslator(), 
)

# 문서 검색
results = retriever.invoke("Find a fund that invests in equities of clean energy production companies.")
results

In [ ]:
for r in results:
    print(r.metadata)

In [ ]:
# ETF의 종목 중에서 첫번째 검색 결과의 symbol을 추출
symbol = results[0].metadata["symbol"]
symbol

In [ ]:
# ETF의 종목 중에서 첫번째 검색 결과의 symbol을 이용하여 Schwab의 ETF 보유 종목을 검색
df_holdings, last_price, date = get_schwab_etf_holdings(symbol)
print(last_price, date)
df_holdings.head()

In [ ]:
# 사용자의 질문을 입력받아서 ETF를 검색하고 Schwab의 ETF 보유 종목을 출력하는 함수를 정의
def get_etf_holdings(query, max_retry=3):
    i = 0
    while i < max_retry:
        try:
            results = retriever.invoke(query)
            symbol = results[0].metadata["symbol"]
            df_holdings, last_price, date = get_schwab_etf_holdings(symbol)
            return df_holdings, last_price, date
        except:
            i += 1
            continue
    return pd.DataFrame(), np.nan, np.nan

In [ ]:
# 함수 실행
query = "Find a fund that invests in equities of clean energy production companies."
df_holdings, last_price, date = get_etf_holdings(query)
print(last_price, date)
df_holdings.head()